In [ ]:
import os
import glob
import numpy as np

path = './traffin_sign_png/'
full_names = os.listdir(path)
labels = sorted([each.split('.')[0] for each in full_names])

In [ ]:
# example of brighting image augmentation
from tqdm.notebook import tqdm
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

import cv2

os.mkdir('./traffic_image')

range_ = tqdm(labels)
for dir_num in range_:
    # 이미지 로드
    img = load_img("./traffin_sign_png/{}.png".format(dir_num))
    # Numpy array 로 변환
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # image data augmentation generator 생성
    datagen = ImageDataGenerator(
        brightness_range=[0.2, 2.0],
        zoom_range=[0.3, 1],
        rotation_range=20,
        height_shift_range=0.2,
        width_shift_range=0.2)
    # prepare iterator
    it = datagen.flow(samples, batch_size=1)
    os.mkdir('./traffic_image/{}'.format(dir_num))

    for i in range(1000):
        batch = it.next()
        image = batch[0].astype("uint8")
        # rgb 변환
        b, g, r = cv2.split(image)
        img_astro3_rgb = cv2.merge([r, g, b])
        cv2.imwrite("./traffic_image/{}/{}_{}.png".format(dir_num,
                                                          dir_num, i), img_astro3_rgb)

In [ ]:
from PIL import Image
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "./traffic_image/"
categories = labels
nb_classes = len(labels)

image_w = 64
image_h = 64

X = []
y = []

for idx, cat in enumerate(categories):

    #   one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + str(cat)
    files = glob.glob(image_dir+"/*.png")
    print(cat, " 파일 길이 : ", len(files))
    # 이미지 파일을 64 x 64 로 줄이고, 벡터화 시켜 X에 저장, one-hot-encoding된 라벨도 저장
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

X = np.array(X)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
X_train.shape

y_train_test = y_train.reshape(-1,1)
y_train_test.shape
y_train.shape

import pickle 
pickle.dump(xy, open("./model/multi_image_data.npy", 'wb'), protocol=4)

In [ ]:
X_train, X_test, y_train, y_test = np.load('./model/multi_image_data.npy',allow_pickle=True)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# 일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

모델링

In [ ]:
import os
import glob
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

nb_classes = len(labels)

with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding="same",
                     input_shape=X_train.shape[1:], activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))

    # 학습을 돌리는 방법을 정의 : cost function을 설정하고, 어떻게 최적화 할건지 방법을 정하고
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    model_dir = './model'

    if not os.path.exists(model_dir):
        os.mkdir(model_dir)

    model_path = model_dir + '/multi_img_classification.model'
    checkpoint = ModelCheckpoint(
        filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.2, callbacks=[checkpoint, early_stopping])

In [ ]:

plot_target = ['loss', 'val_loss', 'accuracy', 'val_accuracy']
for each in plot_target:
    plt.plot(history.history[each], label=each)
    
plt.legend()
plt.show()

틀린것 확인

In [ ]:
from keras.models import load_model
model = load_model('model/multi_img_classification.model')

In [ ]:
y_test[0]

In [ ]:
import numpy as np

predicted_result = model.predict(X_test)
predicted_labels = np.argmax(predicted_result, axis=1)
predicted_labels[:10]

In [ ]:
y_labels = []
for vector in y_test:
    for idx, i in enumerate(vector):
        if i != 0:
            y_labels.append(idx)

In [ ]:
y_labels = np.array(y_labels)
y_labels

In [ ]:
wrong_result = []

for n in range(0, len(y_test)):
    if predicted_labels[n] != y_labels[n]:
        wrong_result.append(n)
        
len(wrong_result)

In [ ]:

import random

samples = random.choices(population=wrong_result, k=4)

In [ ]:
label_to_str = ["+자형교차로","T자형교차로","Y자형교차로","ㅏ자형교차로","ㅓ자형교차로","우선도로","우합류도로","좌합류도로","회전형교차로","철길건널목","우로굽은도로","좌로굽은도로","우좌로이중굽은도로","좌우로이중굽은도로","2방향통행","오르막경사","내리막경사","도로폭이좁아짐","우측차로없어짐","좌측차로없어짐","우측방통행","양측방통행","중앙분리대시작","중앙분리대끝남","신호기","미끄러운도로","강변도로","노면고르지못함","과속방지턱","낙석도로","횡단보도","어린이보호","자전거","도로공사중","비행기","횡풍","터널","교량","야생동물보호","위험","상습정체구간","통행금지","자동차통행금지","화물자동차통행금지","승합자동차통행금지","이륜자동차및원동기장치자전거통행금지","자동차, 이륜자동차빛원동기장치자전거통행금지","경운기, 트렉터및 손수레통행금지","자전거통행금지","진입금지","직진금지","우회전금지","좌회전금지","유턴금지","앞지르기금지","정차,주차금지","주차금지","차중량제한","차높이제한","차폭제한","차간거리확보","최고속도제한","최저속도제한","서행","일시정지","양보","보행자보행금지","위험물적재차량 통행금지"]

In [ ]:

plt.figure(figsize=(14,12))

for idx, n in enumerate(samples):
    plt.subplot(4, 2, idx+1)
    plt.imshow(X_test[n].reshape(64,64,3), cmap='Greys', interpolation='nearest')
    plt.title('Label : ' + label_to_str[y_labels[n]] + ',  Predict : ' + label_to_str[predicted_labels[n]])
    plt.axis('off')

plt.show()

실제 촬영이미지로 테스트

In [ ]:
from keras.models import load_model
model = load_model('model/multi_img_classification.model')

In [ ]:
from PIL import Image

In [ ]:
# 이미지 파일을 64 x 64 로 줄이고, 벡터화 시켜 X에 저장
image_w = 64
image_h = 64
X = []
img = Image.open('test2.jpeg')
img = img.convert("RGB")
img_resized = img.resize((image_w, image_h))
data = np.asarray(img_resized)
X.append(data)
X = np.array(X)
X = X.astype(float) / 255

In [ ]:
img

In [ ]:

result = model.predict(X)
label_to_str[np.argmax(result, axis=1)[0]]

output='좌우로이중굽은도로'